## **Analysis of the library** ***Uncertainnpy***

In [ ]:
# Install and load libraries
!pip install uncertainpy
import uncertainpy as un

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Built in example:**
## A cooling coffee cup model

In [ ]:
import chaospy as cp                       
import numpy as np                         
from scipy.integrate import odeint         

# Create the coffee cup model function
def coffee_cup(kappa, T_env):
    # Initial temperature and time array
    time = np.linspace(0, 200, 150)            # Minutes
    T_0 = 95                                   # Celsius

    # The equation describing the model
    def f(T, time, kappa, T_env):
        return -kappa*(T - T_env)

    # Solving the equation by integration
    temperature = odeint(f, T_0, time, args=(kappa, T_env))[:, 0]

    # Return time and model output
    return time, temperature


# Create a model from the coffee_cup function and add labels
model = un.Model(run=coffee_cup, labels=["Time (min)", "Temperature (C)"])

# Create the distributions
kappa_dist = cp.Uniform(0.025, 0.075)
T_env_dist = cp.Uniform(15, 25)

# Define the parameter dictionary
parameters = {"kappa": kappa_dist, "T_env": T_env_dist}

# Set up the uncertainty quantification
UQ = un.UncertaintyQuantification(model=model, parameters=parameters)

# Perform the uncertainty quantification using
# polynomial chaos with point collocation (by default)
# We set the seed to easier be able to reproduce the result
data = UQ.quantify(seed=10)

# **External code example**:
## The Courtemanche model


> During simulation, differential equations are solved using ode15 with a resolution time series per ms at a fixed pacing cycle length of 1000 ms.

> In particular we have as input a vector of states, which elements are the parameters on which the membrane voltage depends and a vector of constants that are the costants of the model.

> As output, indeed, we set the membrane voltage (V) and the slope of the action potential upstroke (dV/dt) as a functions of time.

## 1.   Definition of the model 


In [ ]:
# Define the size of variable arrays
sizeAlgebraic = 75
sizeStates = 21
sizeConstants = 49

In [ ]:
from math import *
from numpy import *

# Definition of the required function to solve model with ODE solver
def model(C_m, G_Na, Na_o, G_K1, K_o, G_to, G_Kr, G_ks, G_CaL,
          i_NaK_max, Gb_Na, Gb_Ca, Ca2_o, i_NaCa_max, ip_Ca_max, 
          t_tr, i_up_max, K_up):
    
    # Define the initial conditions of the ode and set the costant
    def initConsts():
        constants = [0.0] * sizeConstants; 
        states = [0.0] * sizeStates;

        #Definition of initial condition of the outputs
        states[0] = -80.7943720540656
        states[1] = 11.1978985821874
        states[2] = 0.0030991545302638
        states[3] = 0.961779024084024
        states[4] = 0.975082606579783
        states[5] = 138.966052602725
        states[6] = 0.0310953022800734
        states[7] = 0.999182814284718
        states[8] = 0.00517060631778166
        states[9] = 0.990418619902699
        states[10] = 0.00305781615411067
        states[11] = 0.0200076100020844
        states[12] = 0.000116546960393694
        states[13] = 0.000143471458047068
        states[14] = 0.906829583502354
        states[15] = 0.750035883990791
        states[16] = 1.09657547671994
        states[17] = 2.36683179892234e-50
        states[18] = 0.999999999999989
        states[19] = 0.999180121409289
        states[20] = 1.56333172702071

        #Definition of constant parameters
        constants[0] = 8.3143
        constants[1] = 310
        constants[2] = 96.4867
        constants[3] = C_m         # 100 pF
        constants[4] = 50
        constants[5] = 50000
        constants[6] = 1000
        constants[7] = 2
        constants[8] = -2000
        constants[9] = G_Na         # 7.8 ns/pF
        constants[10] = Na_o        # 140 mMol
        constants[11] = G_K1        # 0.09 ns/pF
        constants[12] = K_o         # 5.4 mMol
        constants[13] = 3
        constants[14] = G_to        # 0.1652 ns/pF
        constants[15] = G_Kr        # 0.029411765 ns/Pf
        constants[16] = G_ks        # 0.12941176 ns/pF
        constants[17] = G_CaL       # 0.12375 ns/pF
        constants[18] = 10
        constants[19] = 1.5
        constants[20] = i_NaK_max   # 0.59933874 pA/pF
        constants[21] = Gb_Na       # 0.0006744375 ns/pF
        constants[22] = Gb_Ca       # 0.001131 ns/pF
        constants[23] = 0
        constants[24] = Ca2_o       # 1.8 mmol
        constants[25] = i_NaCa_max  # 1600 pA/pF
        constants[26] = 87.5
        constants[27] = 1.38
        constants[28] = 0.1
        constants[29] = 0.35
        constants[30] = ip_Ca_max   # 0.275 pA/pF
        constants[31] = 30
        constants[32] = t_tr        # 180 ms
        constants[33] = i_up_max    # 0.005 mmol/ms
        constants[34] = K_up        # 0.00092 mmol
        constants[35] = 15
        constants[36] = 0.05
        constants[37] = 0.07
        constants[38] = 10
        constants[39] = 0.00238
        constants[40] = 0.0005
        constants[41] = 0.8
        constants[42] = 20100
        constants[43] = constants[42]*0.680000
        constants[44] = 2.00000
        constants[45] = (1.00000/7.00000)*(exp(constants[10]/67.3000)-1.00000)
        constants[46] = 8.00000
        constants[47] = 0.00480000*constants[42]
        constants[48] = 0.0552000*constants[42]
        return (states, constants)

    # Computation of rates
    def computeRates(voi, states, constants):
        # Compute result of a piecewise function
        def custom_piecewise(cases):
            return select(cases[0::2],cases[1::2])
        
        rates = [0.0] * sizeStates; 
        
        algebraic = [0.0] * sizeAlgebraic
        algebraic[12] = power(1.00000+states[12]/0.000350000, -1.00000)
        rates[15] = (algebraic[12]-states[15])/constants[44]
        algebraic[10] = power(1.00000+exp((states[0]+10.0000)/-8.00000), -1.00000)
        algebraic[27] = custom_piecewise([less(fabs(states[0]+10.0000) , 1.00000e-10), 4.57900/(1.00000+exp((states[0]+10.0000)/-6.24000)) , True, (1.00000-exp((states[0]+10.0000)/-6.24000))/(0.0350000*(states[0]+10.0000)*(1.00000+exp((states[0]+10.0000)/-6.24000)))])
        rates[13] = (algebraic[10]-states[13])/algebraic[27]
        algebraic[11] = exp(-(states[0]+28.0000)/6.90000)/(1.00000+exp(-(states[0]+28.0000)/6.90000))
        algebraic[28] = 9.00000*(power(0.0197000*exp(-(power(0.0337000, 2.00000))*(power(states[0]+10.0000, 2.00000)))+0.0200000, -1.00000))
        rates[14] = (algebraic[11]-states[14])/algebraic[28]
        algebraic[13] = custom_piecewise([less(fabs(states[0]-7.90000) , 1.00000e-10), (6.00000*0.200000)/1.30000 , True, (6.00000*(1.00000-exp(-(states[0]-7.90000)/5.00000)))/((1.00000+0.300000*exp(-(states[0]-7.90000)/5.00000))*1.00000*(states[0]-7.90000))])
        algebraic[29] = 1.00000-power(1.00000+exp(-(states[0]-40.0000)/17.0000), -1.00000)
        rates[19] = (algebraic[29]-states[19])/algebraic[13]
        algebraic[1] = custom_piecewise([equal(states[0] , -47.1300), 3.20000 , True, (0.320000*(states[0]+47.1300))/(1.00000-exp(-0.100000*(states[0]+47.1300)))])
        algebraic[18] = 0.0800000*exp(-states[0]/11.0000)
        algebraic[31] = algebraic[1]/(algebraic[1]+algebraic[18])
        algebraic[41] = 1.00000/(algebraic[1]+algebraic[18])
        rates[2] = (algebraic[31]-states[2])/algebraic[41]
        algebraic[2] = custom_piecewise([less(states[0] , -40.0000), 0.135000*exp((states[0]+80.0000)/-6.80000) , True, 0.00000])
        algebraic[19] = custom_piecewise([less(states[0] , -40.0000), 3.56000*exp(0.0790000*states[0])+310000.*exp(0.350000*states[0]) , True, 1.00000/(0.130000*(1.00000+exp((states[0]+10.6600)/-11.1000)))])
        algebraic[32] = algebraic[2]/(algebraic[2]+algebraic[19])
        algebraic[42] = 1.00000/(algebraic[2]+algebraic[19])
        rates[3] = (algebraic[32]-states[3])/algebraic[42]
        algebraic[3] = custom_piecewise([less(states[0] , -40.0000), ((-127140.*exp(0.244400*states[0])-3.47400e-05*exp(-0.0439100*states[0]))*(states[0]+37.7800))/(1.00000+exp(0.311000*(states[0]+79.2300))) , True, 0.00000])
        algebraic[20] = custom_piecewise([less(states[0] , -40.0000), (0.121200*exp(-0.0105200*states[0]))/(1.00000+exp(-0.137800*(states[0]+40.1400))) , True, (0.300000*exp(-2.53500e-07*states[0]))/(1.00000+exp(-0.100000*(states[0]+32.0000)))])
        algebraic[33] = algebraic[3]/(algebraic[3]+algebraic[20])
        algebraic[43] = 1.00000/(algebraic[3]+algebraic[20])
        rates[4] = (algebraic[33]-states[4])/algebraic[43]
        algebraic[4] = 0.650000*(power(exp((states[0]--10.0000)/-8.50000)+exp(((states[0]--10.0000)-40.0000)/-59.0000), -1.00000))
        algebraic[21] = 0.650000*(power(2.50000+exp(((states[0]--10.0000)+72.0000)/17.0000), -1.00000))
        algebraic[34] = (power(algebraic[4]+algebraic[21], -1.00000))/constants[13]
        algebraic[44] = power(1.00000+exp(((states[0]--10.0000)+10.4700)/-17.5400), -1.00000)
        rates[6] = (algebraic[44]-states[6])/algebraic[34]
        algebraic[5] = power(18.5300+1.00000*exp(((states[0]--10.0000)+103.700)/10.9500), -1.00000)
        algebraic[22] = power(35.5600+1.00000*exp(((states[0]--10.0000)-8.74000)/-7.44000), -1.00000)
        algebraic[35] = (power(algebraic[5]+algebraic[22], -1.00000))/constants[13]
        algebraic[45] = power(1.00000+exp(((states[0]--10.0000)+33.1000)/5.30000), -1.00000)
        rates[7] = (algebraic[45]-states[7])/algebraic[35]
        algebraic[6] = 0.650000*(power(exp((states[0]--10.0000)/-8.50000)+exp(((states[0]--10.0000)-40.0000)/-59.0000), -1.00000))
        algebraic[23] = 0.650000*(power(2.50000+exp(((states[0]--10.0000)+72.0000)/17.0000), -1.00000))
        algebraic[36] = (power(algebraic[6]+algebraic[23], -1.00000))/constants[13]
        algebraic[46] = power(1.00000+exp(((states[0]--10.0000)+20.3000)/-9.60000), -1.00000)
        rates[8] = (algebraic[46]-states[8])/algebraic[36]
        algebraic[7] = power(21.0000+1.00000*exp(((states[0]--10.0000)-195.000)/-28.0000), -1.00000)
        algebraic[24] = 1.00000/exp(((states[0]--10.0000)-168.000)/-16.0000)
        algebraic[37] = (power(algebraic[7]+algebraic[24], -1.00000))/constants[13]
        algebraic[47] = power(1.00000+exp(((states[0]--10.0000)-109.450)/27.4800), -1.00000)
        rates[9] = (algebraic[47]-states[9])/algebraic[37]
        algebraic[8] = custom_piecewise([less(fabs(states[0]+14.1000) , 1.00000e-10), 0.00150000 , True, (0.000300000*(states[0]+14.1000))/(1.00000-exp((states[0]+14.1000)/-5.00000))])
        algebraic[25] = custom_piecewise([less(fabs(states[0]-3.33280) , 1.00000e-10), 0.000378361 , True, (7.38980e-05*(states[0]-3.33280))/(exp((states[0]-3.33280)/5.12370)-1.00000)])
        algebraic[38] = power(algebraic[8]+algebraic[25], -1.00000)
        algebraic[48] = power(1.00000+exp((states[0]+14.1000)/-6.50000), -1.00000)
        rates[10] = (algebraic[48]-states[10])/algebraic[38]
        algebraic[9] = custom_piecewise([less(fabs(states[0]-19.9000) , 1.00000e-10), 0.000680000 , True, (4.00000e-05*(states[0]-19.9000))/(1.00000-exp((states[0]-19.9000)/-17.0000))])
        algebraic[26] = custom_piecewise([less(fabs(states[0]-19.9000) , 1.00000e-10), 0.000315000 , True, (3.50000e-05*(states[0]-19.9000))/(exp((states[0]-19.9000)/9.00000)-1.00000)])
        algebraic[39] = 0.500000*(power(algebraic[9]+algebraic[26], -1.00000))
        algebraic[49] = power(1.00000+exp((states[0]-19.9000)/-12.7000), -0.500000)
        rates[11] = (algebraic[49]-states[11])/algebraic[39]
        algebraic[40] = ((constants[0]*constants[1])/constants[2])*log(constants[12]/states[5])
        algebraic[50] = (constants[3]*constants[11]*(states[0]-algebraic[40]))/(1.00000+exp(0.0700000*(states[0]+80.0000)))
        algebraic[51] = constants[3]*constants[14]*(power(states[6], 3.00000))*states[7]*(states[0]-algebraic[40])
        algebraic[52] = 0.00500000+0.0500000/(1.00000+exp((states[0]-15.0000)/-13.0000))
        algebraic[53] = constants[3]*algebraic[52]*(power(states[8], 3.00000))*states[9]*(states[0]-algebraic[40])
        algebraic[54] = (constants[3]*constants[15]*states[10]*(states[0]-algebraic[40]))/(1.00000+exp((states[0]+15.0000)/22.4000))
        algebraic[55] = constants[3]*constants[16]*(power(states[11], 2.00000))*(states[0]-algebraic[40])
        algebraic[57] = power(1.00000+0.124500*exp((-0.100000*constants[2]*states[0])/(constants[0]*constants[1]))+0.0365000*constants[45]*exp((-constants[2]*states[0])/(constants[0]*constants[1])), -1.00000)
        algebraic[58] = (((constants[3]*constants[20]*algebraic[57]*1.00000)/(1.00000+power(constants[18]/states[1], 1.50000)))*constants[12])/(constants[12]+constants[19])
        algebraic[60] = constants[3]*constants[23]*(states[0]-algebraic[40])
        rates[5] = (2.00000*algebraic[58]-(algebraic[50]+algebraic[51]+algebraic[53]+algebraic[54]+algebraic[55]+algebraic[60]))/(constants[43]*constants[2])
        algebraic[17] = ((constants[0]*constants[1])/constants[2])*log(constants[10]/states[1])
        algebraic[30] = constants[3]*constants[9]*(power(states[2], 3.00000))*states[3]*states[4]*(states[0]-algebraic[17])
        algebraic[63] = (constants[3]*constants[25]*(exp((constants[29]*constants[2]*states[0])/(constants[0]*constants[1]))*(power(states[1], 3.00000))*constants[24]-exp(((constants[29]-1.00000)*constants[2]*states[0])/(constants[0]*constants[1]))*(power(constants[10], 3.00000))*states[12]))/((power(constants[26], 3.00000)+power(constants[10], 3.00000))*(constants[27]+constants[24])*(1.00000+constants[28]*exp(((constants[29]-1.00000)*states[0]*constants[2])/(constants[0]*constants[1]))))
        algebraic[61] = constants[3]*constants[21]*(states[0]-algebraic[17])
        rates[1] = (-3.00000*algebraic[58]-(3.00000*algebraic[63]+algebraic[61]+algebraic[30]))/(constants[43]*constants[2])
        algebraic[0] = custom_piecewise([greater_equal(voi , constants[4]) & less_equal(voi , constants[5]) & less_equal((voi-constants[4])-floor((voi-constants[4])/constants[6])*constants[6] , constants[7]), constants[8] , True, 0.00000])
        algebraic[56] = constants[3]*constants[17]*states[13]*states[14]*states[15]*(states[0]-65.0000)
        algebraic[64] = (constants[3]*constants[30]*states[12])/(0.000500000+states[12])
        algebraic[59] = ((constants[0]*constants[1])/(2.00000*constants[2]))*log(constants[24]/states[12])
        algebraic[62] = constants[3]*constants[22]*(states[0]-algebraic[59])
        rates[0] = -(algebraic[30]+algebraic[50]+algebraic[51]+algebraic[53]+algebraic[54]+algebraic[55]+algebraic[61]+algebraic[62]+algebraic[58]+algebraic[64]+algebraic[63]+algebraic[56]+algebraic[0])/constants[3]
        algebraic[65] = constants[31]*(power(states[17], 2.00000))*states[18]*states[19]*(states[16]-states[12])
        algebraic[67] = (states[20]-states[16])/constants[32]
        rates[16] = (algebraic[67]-algebraic[65])*(power(1.00000+(constants[38]*constants[41])/(power(states[16]+constants[41], 2.00000)), -1.00000))
        algebraic[66] = 1000.00*(1.00000e-15*constants[47]*algebraic[65]-(1.00000e-15/(2.00000*constants[2]))*(0.500000*algebraic[56]-0.200000*algebraic[63]))
        algebraic[68] = power(1.00000+exp(-(algebraic[66]-3.41750e-13)/1.36700e-15), -1.00000)
        rates[17] = (algebraic[68]-states[17])/constants[46]
        algebraic[69] = 1.91000+2.09000*(power(1.00000+exp(-(algebraic[66]-3.41750e-13)/1.36700e-15), -1.00000))
        algebraic[71] = 1.00000-power(1.00000+exp(-(algebraic[66]-6.83500e-14)/1.36700e-15), -1.00000)
        rates[18] = (algebraic[71]-states[18])/algebraic[69]
        algebraic[70] = constants[33]/(1.00000+constants[34]/states[12])
        algebraic[72] = (constants[33]*states[20])/constants[35]
        rates[20] = algebraic[70]-(algebraic[72]+(algebraic[67]*constants[47])/constants[48])
        algebraic[73] = (2.00000*algebraic[63]-(algebraic[64]+algebraic[56]+algebraic[62]))/(2.00000*constants[43]*constants[2])+(constants[48]*(algebraic[72]-algebraic[70])+algebraic[65]*constants[47])/constants[43]
        algebraic[74] = 1.00000+(constants[37]*constants[40])/(power(states[12]+constants[40], 2.00000))+(constants[36]*constants[39])/(power(states[12]+constants[39], 2.00000))
        rates[12] = algebraic[73]/algebraic[74]
        return(rates)

    # Computation of algebraic
    def computeAlgebraic(constants, states, voi):

        # Compute result of a piecewise function
        def custom_piecewise(cases):
            return select(cases[0::2],cases[1::2])
        
        algebraic = array([[0.0] * len(voi)] * sizeAlgebraic)
        states = array(states)
        voi = array(voi)
        algebraic[12] = power(1.00000+states[12]/0.000350000, -1.00000)
        algebraic[10] = power(1.00000+exp((states[0]+10.0000)/-8.00000), -1.00000)
        algebraic[27] = custom_piecewise([less(fabs(states[0]+10.0000) , 1.00000e-10), 4.57900/(1.00000+exp((states[0]+10.0000)/-6.24000)) , True, (1.00000-exp((states[0]+10.0000)/-6.24000))/(0.0350000*(states[0]+10.0000)*(1.00000+exp((states[0]+10.0000)/-6.24000)))])
        algebraic[11] = exp(-(states[0]+28.0000)/6.90000)/(1.00000+exp(-(states[0]+28.0000)/6.90000))
        algebraic[28] = 9.00000*(power(0.0197000*exp(-(power(0.0337000, 2.00000))*(power(states[0]+10.0000, 2.00000)))+0.0200000, -1.00000))
        algebraic[13] = custom_piecewise([less(fabs(states[0]-7.90000) , 1.00000e-10), (6.00000*0.200000)/1.30000 , True, (6.00000*(1.00000-exp(-(states[0]-7.90000)/5.00000)))/((1.00000+0.300000*exp(-(states[0]-7.90000)/5.00000))*1.00000*(states[0]-7.90000))])
        algebraic[29] = 1.00000-power(1.00000+exp(-(states[0]-40.0000)/17.0000), -1.00000)
        algebraic[1] = custom_piecewise([equal(states[0] , -47.1300), 3.20000 , True, (0.320000*(states[0]+47.1300))/(1.00000-exp(-0.100000*(states[0]+47.1300)))])
        algebraic[18] = 0.0800000*exp(-states[0]/11.0000)
        algebraic[31] = algebraic[1]/(algebraic[1]+algebraic[18])
        algebraic[41] = 1.00000/(algebraic[1]+algebraic[18])
        algebraic[2] = custom_piecewise([less(states[0] , -40.0000), 0.135000*exp((states[0]+80.0000)/-6.80000) , True, 0.00000])
        algebraic[19] = custom_piecewise([less(states[0] , -40.0000), 3.56000*exp(0.0790000*states[0])+310000.*exp(0.350000*states[0]) , True, 1.00000/(0.130000*(1.00000+exp((states[0]+10.6600)/-11.1000)))])
        algebraic[32] = algebraic[2]/(algebraic[2]+algebraic[19])
        algebraic[42] = 1.00000/(algebraic[2]+algebraic[19])
        algebraic[3] = custom_piecewise([less(states[0] , -40.0000), ((-127140.*exp(0.244400*states[0])-3.47400e-05*exp(-0.0439100*states[0]))*(states[0]+37.7800))/(1.00000+exp(0.311000*(states[0]+79.2300))) , True, 0.00000])
        algebraic[20] = custom_piecewise([less(states[0] , -40.0000), (0.121200*exp(-0.0105200*states[0]))/(1.00000+exp(-0.137800*(states[0]+40.1400))) , True, (0.300000*exp(-2.53500e-07*states[0]))/(1.00000+exp(-0.100000*(states[0]+32.0000)))])
        algebraic[33] = algebraic[3]/(algebraic[3]+algebraic[20])
        algebraic[43] = 1.00000/(algebraic[3]+algebraic[20])
        algebraic[4] = 0.650000*(power(exp((states[0]--10.0000)/-8.50000)+exp(((states[0]--10.0000)-40.0000)/-59.0000), -1.00000))
        algebraic[21] = 0.650000*(power(2.50000+exp(((states[0]--10.0000)+72.0000)/17.0000), -1.00000))
        algebraic[34] = (power(algebraic[4]+algebraic[21], -1.00000))/constants[13]
        algebraic[44] = power(1.00000+exp(((states[0]--10.0000)+10.4700)/-17.5400), -1.00000)
        algebraic[5] = power(18.5300+1.00000*exp(((states[0]--10.0000)+103.700)/10.9500), -1.00000)
        algebraic[22] = power(35.5600+1.00000*exp(((states[0]--10.0000)-8.74000)/-7.44000), -1.00000)
        algebraic[35] = (power(algebraic[5]+algebraic[22], -1.00000))/constants[13]
        algebraic[45] = power(1.00000+exp(((states[0]--10.0000)+33.1000)/5.30000), -1.00000)
        algebraic[6] = 0.650000*(power(exp((states[0]--10.0000)/-8.50000)+exp(((states[0]--10.0000)-40.0000)/-59.0000), -1.00000))
        algebraic[23] = 0.650000*(power(2.50000+exp(((states[0]--10.0000)+72.0000)/17.0000), -1.00000))
        algebraic[36] = (power(algebraic[6]+algebraic[23], -1.00000))/constants[13]
        algebraic[46] = power(1.00000+exp(((states[0]--10.0000)+20.3000)/-9.60000), -1.00000)
        algebraic[7] = power(21.0000+1.00000*exp(((states[0]--10.0000)-195.000)/-28.0000), -1.00000)
        algebraic[24] = 1.00000/exp(((states[0]--10.0000)-168.000)/-16.0000)
        algebraic[37] = (power(algebraic[7]+algebraic[24], -1.00000))/constants[13]
        algebraic[47] = power(1.00000+exp(((states[0]--10.0000)-109.450)/27.4800), -1.00000)
        algebraic[8] = custom_piecewise([less(fabs(states[0]+14.1000) , 1.00000e-10), 0.00150000 , True, (0.000300000*(states[0]+14.1000))/(1.00000-exp((states[0]+14.1000)/-5.00000))])
        algebraic[25] = custom_piecewise([less(fabs(states[0]-3.33280) , 1.00000e-10), 0.000378361 , True, (7.38980e-05*(states[0]-3.33280))/(exp((states[0]-3.33280)/5.12370)-1.00000)])
        algebraic[38] = power(algebraic[8]+algebraic[25], -1.00000)
        algebraic[48] = power(1.00000+exp((states[0]+14.1000)/-6.50000), -1.00000)
        algebraic[9] = custom_piecewise([less(fabs(states[0]-19.9000) , 1.00000e-10), 0.000680000 , True, (4.00000e-05*(states[0]-19.9000))/(1.00000-exp((states[0]-19.9000)/-17.0000))])
        algebraic[26] = custom_piecewise([less(fabs(states[0]-19.9000) , 1.00000e-10), 0.000315000 , True, (3.50000e-05*(states[0]-19.9000))/(exp((states[0]-19.9000)/9.00000)-1.00000)])
        algebraic[39] = 0.500000*(power(algebraic[9]+algebraic[26], -1.00000))
        algebraic[49] = power(1.00000+exp((states[0]-19.9000)/-12.7000), -0.500000)
        algebraic[40] = ((constants[0]*constants[1])/constants[2])*log(constants[12]/states[5])
        algebraic[50] = (constants[3]*constants[11]*(states[0]-algebraic[40]))/(1.00000+exp(0.0700000*(states[0]+80.0000)))
        algebraic[51] = constants[3]*constants[14]*(power(states[6], 3.00000))*states[7]*(states[0]-algebraic[40])
        algebraic[52] = 0.00500000+0.0500000/(1.00000+exp((states[0]-15.0000)/-13.0000))
        algebraic[53] = constants[3]*algebraic[52]*(power(states[8], 3.00000))*states[9]*(states[0]-algebraic[40])
        algebraic[54] = (constants[3]*constants[15]*states[10]*(states[0]-algebraic[40]))/(1.00000+exp((states[0]+15.0000)/22.4000))
        algebraic[55] = constants[3]*constants[16]*(power(states[11], 2.00000))*(states[0]-algebraic[40])
        algebraic[57] = power(1.00000+0.124500*exp((-0.100000*constants[2]*states[0])/(constants[0]*constants[1]))+0.0365000*constants[45]*exp((-constants[2]*states[0])/(constants[0]*constants[1])), -1.00000)
        algebraic[58] = (((constants[3]*constants[20]*algebraic[57]*1.00000)/(1.00000+power(constants[18]/states[1], 1.50000)))*constants[12])/(constants[12]+constants[19])
        algebraic[60] = constants[3]*constants[23]*(states[0]-algebraic[40])
        algebraic[17] = ((constants[0]*constants[1])/constants[2])*log(constants[10]/states[1])
        algebraic[30] = constants[3]*constants[9]*(power(states[2], 3.00000))*states[3]*states[4]*(states[0]-algebraic[17])
        algebraic[63] = (constants[3]*constants[25]*(exp((constants[29]*constants[2]*states[0])/(constants[0]*constants[1]))*(power(states[1], 3.00000))*constants[24]-exp(((constants[29]-1.00000)*constants[2]*states[0])/(constants[0]*constants[1]))*(power(constants[10], 3.00000))*states[12]))/((power(constants[26], 3.00000)+power(constants[10], 3.00000))*(constants[27]+constants[24])*(1.00000+constants[28]*exp(((constants[29]-1.00000)*states[0]*constants[2])/(constants[0]*constants[1]))))
        algebraic[61] = constants[3]*constants[21]*(states[0]-algebraic[17])
        algebraic[0] = custom_piecewise([greater_equal(voi , constants[4]) & less_equal(voi , constants[5]) & less_equal((voi-constants[4])-floor((voi-constants[4])/constants[6])*constants[6] , constants[7]), constants[8] , True, 0.00000])
        algebraic[56] = constants[3]*constants[17]*states[13]*states[14]*states[15]*(states[0]-65.0000)
        algebraic[64] = (constants[3]*constants[30]*states[12])/(0.000500000+states[12])
        algebraic[59] = ((constants[0]*constants[1])/(2.00000*constants[2]))*log(constants[24]/states[12])
        algebraic[62] = constants[3]*constants[22]*(states[0]-algebraic[59])
        algebraic[65] = constants[31]*(power(states[17], 2.00000))*states[18]*states[19]*(states[16]-states[12])
        algebraic[67] = (states[20]-states[16])/constants[32]
        algebraic[66] = 1000.00*(1.00000e-15*constants[47]*algebraic[65]-(1.00000e-15/(2.00000*constants[2]))*(0.500000*algebraic[56]-0.200000*algebraic[63]))
        algebraic[68] = power(1.00000+exp(-(algebraic[66]-3.41750e-13)/1.36700e-15), -1.00000)
        algebraic[69] = 1.91000+2.09000*(power(1.00000+exp(-(algebraic[66]-3.41750e-13)/1.36700e-15), -1.00000))
        algebraic[71] = 1.00000-power(1.00000+exp(-(algebraic[66]-6.83500e-14)/1.36700e-15), -1.00000)
        algebraic[70] = constants[33]/(1.00000+constants[34]/states[12])
        algebraic[72] = (constants[33]*states[20])/constants[35]
        algebraic[73] = (2.00000*algebraic[63]-(algebraic[64]+algebraic[56]+algebraic[62]))/(2.00000*constants[43]*constants[2])+(constants[48]*(algebraic[72]-algebraic[70])+algebraic[65]*constants[47])/constants[43]
        algebraic[74] = 1.00000+(constants[37]*constants[40])/(power(states[12]+constants[40], 2.00000))+(constants[36]*constants[39])/(power(states[12]+constants[39], 2.00000))
        algebraic[14] = (constants[36]*states[12])/(states[12]+constants[39])
        algebraic[15] = (constants[37]*states[12])/(states[12]+constants[40])
        algebraic[16] = (constants[38]*states[16])/(states[16]+constants[41])
        return algebraic
    
    from scipy.integrate import ode

    # Initialise constants and state variables
    (init_states, constants) = initConsts()

    # Set timespan to solve over
    time = arange(0, T_end, dt)

    # Construct ODE object to solve
    r = ode(computeRates)
    r.set_integrator('vode', method='bdf', atol=1e-06, rtol=1e-06, max_step=1)
    r.set_initial_value(init_states, time[0])
    r.set_f_params(constants)

    # Solve model
    states = array([[0.0] * len(time)] * sizeStates)
    states[:,0] = init_states
    for (i,t) in enumerate(time[1:]):
        if r.successful():
            r.integrate(t)
            states[:,i+1] = r.y
        else:
            break

    rates = computeRates(time, states, constants)

    # Compute the membrane voltage as time function
    V_m = states[0, :]

    # Compute the slope of the action potential upstroke
    dVdt = rates[0]

    # Define the info dictionary
    info = {"dVdt" : dVdt}
    
    # Return time, model output and info dictionary
    return time, V_m, info



### 1.1. Testing the model with one simulation



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set all the parameters to them theorical values
Cm          = 100
g_Na        = 7.8
Na_o        = 140
g_K1        = 0.09
k_o         = 5.4
G_to        = 0.1652
G_Kr        = 0.029411765
G_ks        = 0.12941176
G_CaL       = 0.12375
i_NaK_max   = 0.59933874
Gb_Na       = 0.0006744375
Gb_Ca       = 0.001131
Ca2_o       = 1.8
i_NaCa_max  = 1600.0
ip_Ca_max   = 0.275
t_tr        = 180.0
i_up_max    = 0.005
K_up        = 0.00092

# Set the time constants
T_end = 1000
dt = 1

# Solve the model 
time, V_m, info = model(Cm, g_Na, Na_o, g_K1, k_o, G_to, G_Kr, G_ks, G_CaL,
                        i_NaK_max, Gb_Na, Gb_Ca, Ca2_o, i_NaCa_max, ip_Ca_max,
                        t_tr, i_up_max, K_up)

# Compute the peak voltage of the action potential
V_max = max(V_m)

# Compute the resting membrane potential 
RestV_m = min(V_m)

# Compute the action potential durations at 50% of repolarization
size = min(V_m) + max(V_m)
for i in range(len(V_m)):
  if V_m[i] >= size * 0.50:
    APD50 = i
  if V_m[i] >= size * 0.90:
    APD90 = i

# Compute the membrane voltage measured at 20%, 40%, 60% and 80% of APD90
t_20 = int(APD90*0.20)
V_20 = V_m[t_20]

t_40 = int(APD90*0.40)
V_40 = V_m[t_40]

t_60 = int(APD90*0.60)
V_60 = V_m[t_60]

t_80 = int(APD90*0.80)
V_80 = V_m[t_80]


In [ ]:
# Plotting the result
plt.plot(time, V_m, color='red')
plt.plot(t_20, V_20, 'k.')
plt.plot(t_40, V_40, 'k.')
plt.plot(t_60, V_60, 'k.')
plt.plot(t_80, V_80, 'k.')
plt.plot(0, RestV_m, 'k.')
plt.plot(50, V_max, 'k.')

plt.plot([0, APD50], [size*0.50, size*0.50], 'b,--' )
plt.plot([0, APD90], [size*0.90, size*0.90], 'b,--' )
plt.title("Plot of the membrane voltage and teorical biomarker")
plt.xlabel("time (ms)")
plt.ylabel("Voltage (mV)")
plt.grid(True, linewidth=0.5,color='white',linestyle='-')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
figure = plt.show()


## 2.   Define the parameter dictionary and the features list
> We set all parameters to have a uniform distribution within a certain interval (known) around their fixed (theoretical) values and compute the features with respect to the desired biomarker


### 2.1 Define the parameter dictionary

In [ ]:
#Definition of constant parameters
parameters = {"G_Na"         :   7.8,
              "C_m"          :   100,
              "Na_o"         :   140, 
              "G_K1"         :   0.09,               
              "K_o"          :   5.4, 
              "G_to"         :   0.1652,          
              "G_Kr"         :   0.029411765,        
              "G_ks"         :   0.12941176,         
              "G_CaL"        :   0.12375, 
              "i_NaK_max"    :   0.59933874,        
              "Gb_Na"        :   0.0006744375,       
              "Gb_Ca"        :   0.001131,      
              "Ca2_o"        :   1.8,                
              "i_NaCa_max"   :   1600.0,   
              "ip_Ca_max"    :   0.275,       
              "t_tr"         :   180.0,               
              "i_up_max"     :   0.005,              
              "K_up"         :   0.00092
             }
                                    
# Create the parameters
parameters = un.Parameters(parameters)

# Set all parameters to have a uniform distribution within a certain
# interval around their fixed value
parameters.set_distribution("G_K1",         un.uniform(0.25))
parameters.set_distribution("G_to",         un.uniform(0.50))
parameters.set_distribution("G_CaL",        un.uniform(0.50))
parameters.set_distribution("Gb_Ca",        un.uniform(0.50))
parameters.set_distribution("i_NaK_max",    un.uniform(0.50))
parameters.set_distribution("G_Kr",         un.uniform(0.50))

# Removed parameters according to the paper
# parameters.set_distribution("G_Na",       un.uniform(0.50))
# parameters.set_distribution("Na_o",       un.uniform(0.10))
# parameters.set_distribution("K_o",        un.uniform(0.10))
# parameters.set_distribution("G_ks",       un.uniform(0.50))
# parameters.set_distribution("Gb_Na",      un.uniform(0.50))
# parameters.set_distribution("i_NaCa_max", un.uniform(0.50))
# parameters.set_distribution("ip_Ca_max",  un.uniform(0.50))
# parameters.set_distribution("t_tr",       un.uniform(0.50))
# parameters.set_distribution("i_up_max",   un.uniform(0.50))
# parameters.set_distribution("K_up",       un.uniform(0.50))
# parameters.set_distribution("C_m",        un.uniform(0.25))
# parameters.set_distribution("Ca2_o",      un.uniform(0.10))

### 2.2 Define the features list

In [ ]:
# Compute the derivative of the V_m
def dVdt(time, V_m, info):
    return time, info["dVdt"]

# Compute the peak voltage of the action potential
def V_max(time, V_m, info):
    return None, max(V_m)

# Compute the peak slope of the action potential upstroke
def dV_max(time, V_m, info):
    return None, max(info["dVdt"])

# Compute the resting membrane potential 
def RestV_m(time, V_m, info):
    return None, min(V_m)

# Compute the action potential durations at 50% of repolarization
def APD50(time, V_m, info):
    size = min(V_m) + max(V_m)
    for i in range(len(V_m)):
      if V_m[i] >= size * 0.50:
        APD50 = i
    return None, APD50

# Compute the action potential durations at 50% of repolarization
def APD90(time, V_m, info):
    size = min(V_m) + max(V_m)
    for i in range(len(V_m)):
      if V_m[i] >= size * 0.90:
        APD90 = i
    return None, APD90

# Compute the membrane voltage measured at 20%, 40%, 60% and 80% of APD90
def V_20(time, V_m, info): 
    t = int(APD90(time, V_m, info)[1]*0.20)
    return t, V_m[t]

def V_40(time, V_m, info):
    t = int(APD90(time, V_m, info)[1]*0.40)
    return t, V_m[t]

def V_60(time, V_m, info):
    t = int(APD90(time, V_m, info)[1]*0.60)
    return t, V_m[t]

def V_80(time, V_m, info):
    t = int(APD90(time, V_m, info)[1]*0.80)
    return t, V_m[t]

# By selecting different combinations of variables in the following vector we 
# establish which outputs we consider in the analysis

features = [dVdt, V_max, dV_max, RestV_m, APD50, APD90, V_20, V_40, V_60, V_80]


## 3. Perform the uncertainty quantification


In [ ]:
T_end = 1000
dt = 1

model = un.Model(run = model)

UQ = un.UncertaintyQuantification(model = model,
                                  parameters = parameters,
                                  features = features)

data = UQ.quantify(method = 'pc', CPUs = 'max', rosenblatt = 'auto',
                   polynomial_order = 3, nr_pc_mc_samples = 1000,
                   save = True, 
                   data_folder = 'drive/MyDrive/output', 
                   filename = 'Analisi_V_e_dV')


### 3.1 Addictional graphical representation of the results

In [ ]:
import seaborn as sb 

# Heat map of the first order Sobol indices
sobol_first = [data["dV_max"].sobol_first, data["V_max"].sobol_first, 
                 data["V_20"].sobol_first,  data["V_40"].sobol_first, 
                 data["V_60"].sobol_first,  data["V_80"].sobol_first, 
                data["APD50"].sobol_first, data["APD90"].sobol_first, 
              data["RestV_m"].sobol_first]

heat_map = sb.heatmap(sobol_first, annot=True, fmt=".2f", cmap="Blues", 
                      xticklabels = [ "G_K1", "G_to", "G_Kr", "G_CaL", 
                                     "i_NaK_max", "Gb_Ca"], 
                      yticklabels = ["dV_max", "V_max", "V_20", "V_40", 
                                     "V_60", "V_80", "APD50", "APD90", 
                                     "RestV_m"],
                      vmin=0, vmax=1)


In [ ]:
import seaborn as sb 

# Heat map of the total Sobol indices
sobol_total = [data["dV_max"].sobol_total, data["V_max"].sobol_total, 
                 data["V_20"].sobol_total,  data["V_40"].sobol_total, 
                 data["V_60"].sobol_total,  data["V_80"].sobol_total, 
                data["APD50"].sobol_total, data["APD90"].sobol_total, 
              data["RestV_m"].sobol_total]

heat_map = sb.heatmap(sobol_total, annot=True, fmt=".2f", cmap = "YlOrBr",  
                      xticklabels = [ "G_K1", "G_to", "G_Kr", "G_CaL", 
                                     "i_NaK_max", "Gb_Ca"], 
                      yticklabels = ["dV_max", "V_max", "V_20", "V_40", 
                                     "V_60", "V_80", "APD50", "APD90", 
                                     "RestV_m"],
                      vmin=0, vmax=1)


Plot of the first order and total Sobol indices with respect time (ms)

In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_K1 first order Sobol index with respect to time
plt.plot(time, data["model"].sobol_first[0], '#EB718B', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_to fisrt order Sobol index with respect to time
plt.plot(time, data["model"].sobol_first[1], '#E3D917', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--',linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_Kr fisrt order Sobol index with respect to time
plt.plot(time, data["model"].sobol_first[2], '#72CE52', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_CaL fisrt order Sobol index with respect to time
plt.plot(time, data["model"].sobol_first[3], '#5EB4B2', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of i_NaK_max fisrt order Sobol index with respect to time
plt.plot(time, data["model"].sobol_first[4], '#3B9EE9', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of Gb_Ca fisrt order Sobol index with respect to time
plt.plot(time, data["model"].sobol_first[5], '#D248EA', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--',linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_K1 total Sobol index with respect to time
plt.plot(time, data["model"].sobol_total[0], '#EB718B', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_to total Sobol index with respect to time
plt.plot(time, data["model"].sobol_total[1], '#E3D917', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--',linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_Kr total Sobol index with respect to time
plt.plot(time, data["model"].sobol_total[2], '#72CE52', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of G_CaL total Sobol index with respect to time
plt.plot(time, data["model"].sobol_total[3], '#5EB4B2', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of i_NaK_max total Sobol index with respect to time
plt.plot(time, data["model"].sobol_total[4], '#3B9EE9', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--', linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
time = range(T_end)

# Plot of Gb_Ca total Sobol index with respect to time
plt.plot(time, data["model"].sobol_total[5], '#D248EA', linewidth=3.0)
plt.plot([50, 50], [0, 1], 'y--',linewidth=2.0)
plt.ylim(top = 1)
plt.ylim(bottom = 0)
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
plt.show()


## 4. Particular analysis on the V_max case

### 4.1 V_max indices computed with Uncertainpy

#### Computation of the indices

In [ ]:
# Since we want to focus on V_max we can reduce the time interval
T_end = 400
dt = 0.5

model = un.Model(run = model)

UQ = un.UncertaintyQuantification(model = model,
                                  parameters = parameters,
                                  features = V_max)

data = UQ.quantify(method = 'pc', CPUs = 'max', rosenblatt = 'auto',
                   polynomial_order = 3, nr_pc_mc_samples = 1000, plot = None)


#### Graphical representation of the indexes

In [ ]:
import seaborn as sb 

data_plot = [data["V_max"].sobol_first, data["V_max"].sobol_total]

# Heat map of V_max Sobol indices
heat_map = sb.heatmap(data_plot, annot=True, fmt=".2f", cmap="YlOrBr", 
                      xticklabels = [ "G_K1", "G_to", "G_Kr", "G_CaL", 
                                     "i_NaK_max", "Gb_Ca"], 
                      yticklabels = ["Sobol_first", "Sobol_total"],
                      vmin=0, vmax=1)

### 4.2 V_max indices computed by hands
> By using Saltelli method we generate the input matrix of the selected parameters before passing it to the Matlab model



In [ ]:
!pip install SALib
from SALib.analyze import sobol
from SALib.sample import saltelli

#### Computation of the indices

In [ ]:
import pandas as pd

# Compute Sobol indexes for all biomarkers with 512 iterations
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_1024iteration.xls')
evaluations_V_max = df.to_numpy()[:, 1]

In [ ]:
# Definition of the problem
problem = {'num_vars' : 6,
           'names'    : ["G_K1"     , 
                         "G_to"     ,
                         "G_Kr"     ,
                         "G_CaL"    , 
                         "i_NaK_max",  
                         "Gb_Ca"    ],
           'bounds'    : [[0.018,  0.045],
                          [0.033, 0.0826],
                          [0.0147, 0.0441],
                          [0.0619, 0.1856],
                          [0.2997, 0.8990],
                          [0.0005, 0.0017]]
          }


In [ ]:
sobol_first_V_max = []
sobol_total_V_max = []

si = sobol.analyze(problem, evaluations_V_max, calc_second_order=False, 
                   num_resamples = 1024)
for i in range(6):
  sobol_first_V_max.append(si["S1"][i])
  sobol_total_V_max.append(si["ST"][i])


#### Graphical representation of the indexes

In [ ]:
import seaborn as sb 

# Heatmap of First Order and Total Sobol Index with respect to the input parameters
sobol = [sobol_first_V_max, sobol_total_V_max]

heat_map = sb.heatmap(sobol, annot=True, fmt=".2f", cmap="YlOrBr", 
                      xticklabels = [ "G_K1", "G_to", "G_Kr", "G_CaL", 
                                     "i_NaK_max", "Gb_Ca"], 
                      yticklabels = ["Sobol_first", "Sobol_total"],
                      vmin=0, vmax=1)

##5. Convergence of the Sobol indexes by using SaLib (used by Uncertainpy)


### 5.1 Import the data

In [ ]:
!pip install SALib
from SALib.analyze import sobol
from SALib.sample import saltelli

In [ ]:
from matplotlib.scale import get_scale_names
import pandas as pd
import numpy as np

first_G_K1 = []
total_G_K1 = []

first_G_to = []
total_G_to = []

first_G_CaL = []
total_G_CaL = []

first_Gb_Ca = []
total_Gb_Ca = []

first_i_NaK_max = []
total_i_NaK_max = []

first_G_Kr = []
total_G_Kr = []


In [ ]:
# Definition of the problem
problem = {'num_vars' : 6,
           'names'    : ["G_K1"     , 
                         "G_to"     ,
                         "G_Kr"     ,
                         "G_CaL"    , 
                         "i_NaK_max",  
                         "Gb_Ca"    ],
           'bounds'    : [[0.018,  0.045],
                          [0.033, 0.0826],
                          [0.0147, 0.0441],
                          [0.0619, 0.1856],
                          [0.2997, 0.8990],
                          [0.0005, 0.0017]]
          }


- Import evaluations with 64 iterations




In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_64iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 64)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

- Import evaluations with 128 iterations

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_128iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 128)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

- Import evaluations with 256 iterations


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_256iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 256)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

- Import evaluations with 512 iterations

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_512iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 512)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

- Import evaluations with 1024 iterations

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_1024iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 1024)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

 - Import evaluations with 2048 iterations

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_2048iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 2048)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

- Import evaluations with 4096 iterations

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Computational_statistics/BIOMARKER_VALUES_4096iteration.xls')
evaluations = df.to_numpy()[:, 1]
si = sobol.analyze(problem, evaluations, calc_second_order=False, 
                   num_resamples = 4096)

first_G_K1.append(si["S1"][0])
total_G_K1.append(si["ST"][0])

first_G_to.append(si["S1"][1])
total_G_to.append(si["ST"][1])

first_G_CaL.append(si["S1"][3])
total_G_CaL.append(si["ST"][3])

first_Gb_Ca.append(si["S1"][5])
total_Gb_Ca.append(si["ST"][5])

first_i_NaK_max.append(si["S1"][4])
total_i_NaK_max.append(si["ST"][4])

first_G_Kr.append(si["S1"][2])
total_G_Kr.append(si["ST"][2])

### 5.2 Plot of the results

In [ ]:
import matplotlib.pyplot as plt

N = [64, 128, 256, 512, 1024, 2048, 4096]

plt.title('Convergence of First Order Sobol indices')
plt.plot(N, first_G_K1, label = 'G_K1')
plt.plot(N,first_G_to, label = 'G_to')
plt.plot(N,first_G_CaL, label = 'G_CaL')
plt.plot(N,first_Gb_Ca, label = 'Gb_Ca')
plt.plot(N,first_i_NaK_max, label = 'i_NaKmax')
plt.plot(N,first_G_Kr, label = 'G_Kr')
plt.legend()
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')

In [ ]:
N = [64, 128, 256, 512, 1024, 2048, 4096]

plt.title('Convergence of Total Order Sobol indices')
plt.plot(N, total_G_K1,label = 'G_K1')
plt.plot(N,total_G_to, label = 'G_to')
plt.plot(N,total_G_CaL, label = 'G_CaL')
plt.plot(N,total_Gb_Ca, label = 'Gb_Ca')
plt.plot(N,total_i_NaK_max, label = 'i_NaKmax')
plt.plot(N,total_G_Kr, label = 'G_Kr')
plt.legend()
plt.grid(color='white')
ax = plt.gca()
ax.set_facecolor('#EAEAF2')
